In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
api_token = os.getenv("AIPROXY_TOKEN")

In [2]:
import requests

In [17]:
import pandas as pd
todo_df=pd.DataFrame(columns=['tasks','duration','deadline'])

In [3]:
headers={
        "Authorization": f"Bearer {api_token}",
        "Content-Type": "application/json"
}

In [4]:
url="https://aiproxy.sanand.workers.dev/openai/v1/chat/completions"

In [ ]:
prompt="""
list of tasks to do 
do my project (duration 3hrs)
complete my assignments(duration 12hrs)
complete my capstone (duration 8 hrs)
bath (30 mins)

the final date is 15th feb
"""

In [8]:
to_list=[
    {
        "task":"Project","duration":3,
    },
    {
        "task":"Assignment","duration":12
    },
    {
        "task":"capstone","duration":8
    },
    {
        "task":"bath","duration":0.5
    }
]
deadline="15 th feb"


In [15]:
def todo(to_list,deadline):
    total_dur=0
    for todo in to_list:
        task=todo['task']
        duration=todo['duration']
        total_dur+=duration

        new_todo = {
            "tasks": task,
            "duration": duration,
            "deadline": deadline
        }
        global todo_df
        todo_df=pd.concat([todo_df,pd.DataFrame(new_todo,index=[0])],ignore_index=True,axis=0)
    return


In [18]:
todo(to_list,deadline)
todo_df

,tasks,duration,deadline
0,Project,3,15 th feb
1,Assignment,12,15 th feb
2,capstone,8,15 th feb
3,bath,0.5,15 th feb


In [ ]:
todo_function_py = [
    {
    "type": "function",
    "function": {
        "name": "get_list",
        "description": "from the given prompt extract the json schema , if required values are missing request the parameters",
        "parameters": {
            "type": "object",
            "properties": {
                "tasks": {
                    "type": "array",
                    "description": "A list of tasks with their respective durations.",
                    "items": {
                        "type": "object",
                        "properties": {
                            "task": {
                                "type": "string",
                                "description": "The name of the task to be completed."
                            },
                            "duration": {
                                "type": "number",
                                "description": "The time required to complete the task (in hours)."
                            }
                        },
                        "required": ["task", "duration"],
                        "additionalProperties": False  # Correct in Python
                    }
                },
                "deadline": {
                    "type": "string",
                    "description": "The deadline for completing all tasks (e.g., '15th Feb')."
                }
            },
            "required": ["tasks", "deadline"],
            "additionalProperties": False  # Correct in Python
        }
    }
    },
    {
        "type": "function",
        "function": {
            "name": "count_day_occurrence",
            "description": "Count occurrences of a specified day from dates in a file.",
            "parameters": {
                "type": "object",
                "properties": {
                    "input_file": {"type": "string", "description": "Input file with dates."},
                    "day": {"type": "string", "description": "Day to count."},
                    "output_file": {"type": "string", "description": "Output file for count."}
                },
                "required": ["input_file", "day", "output_file"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "run_npx_package",
            "description": "Run npx command with optional arg and output file.",
            "parameters": {
                "type": "object",
                "properties": {
                    "package": {"type": "string"},
                    "arg": {"type": "string"},
                    "output_file": {"type": "string"}
                },
                "required": ["package", "output_file"],
                "additionalProperties": False
            }
        }
]



[
    {
        "type": "function",
        "function": {
            "name": "get_list",
            "description": "Extract the json schema",
            "parameters": {
                "type": "object",
                "properties": {
                    "tasks": {
                        "type": "array",
                        "description": "A list of tasks with their respective durations.",
                        "items": {
                            "type": "object",
                            "properties": {
                                "task": {
                                    "type": "string",
                                    "description": "The name of the task to be completed."
                                },
                                "duration": {
                                    "type": "number",
                                    "description": "The time required to complete the task (in hours)."
                                }
              

In [48]:
response=requests.post(url=url,headers=headers, json={
    "model":"gpt-4o-mini",
    "messages":[{
        "role":'user',
        'content':prompt,
    }],
        'tools':todo_function_py,
        'tool_choice':'auto'
}
              )

In [44]:
payload = {
    "model": "gpt-4o-mini", 
    "messages": [{
        "role": "user",
        "content": prompt
    }],
    "tools": todo_function_py,  
    "tool_choice": "auto"  
}

In [45]:
response = requests.post(url=url, headers=headers, json=payload)

In [46]:
response.json()


{'id': 'chatcmpl-Az9nwvOPdyGr2AaJclNrUVkIMHV97',
 'object': 'chat.completion',
 'created': 1739139060,
 'model': 'gpt-4o-mini-2024-07-18',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': None,
    'tool_calls': [{'id': 'call_vmQsM0WMWuXrpZcMxILkS8Ni',
      'type': 'function',
      'function': {'name': 'get_list',
       'arguments': '{"tasks":[{"task":"do my project","duration":3},{"task":"complete my assignments","duration":12},{"task":"complete my capstone","duration":8},{"task":"bath","duration":0.5}],"deadline":"15th Feb"}'}}],
    'refusal': None},
   'logprobs': None,
   'finish_reason': 'tool_calls'}],
 'usage': {'prompt_tokens': 156,
  'completion_tokens': 62,
  'total_tokens': 218,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 0,
   'audio_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0}},
 'service_tier': 'default',
 'system_fingerprint':

In [49]:
response.json()


{'id': 'chatcmpl-Az9rubnikASMnhq9W6oPIhSS77i8j',
 'object': 'chat.completion',
 'created': 1739139306,
 'model': 'gpt-4o-mini-2024-07-18',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': None,
    'tool_calls': [{'id': 'call_8nSQH8nV7tCZZJa1FYowIu9X',
      'type': 'function',
      'function': {'name': 'get_list',
       'arguments': '{"tasks":[{"task":"do my project","duration":3},{"task":"complete my assignments","duration":12},{"task":"complete my capstone","duration":8},{"task":"bath","duration":0.5}],"deadline":"15th Feb"}'}}],
    'refusal': None},
   'logprobs': None,
   'finish_reason': 'tool_calls'}],
 'usage': {'prompt_tokens': 156,
  'completion_tokens': 62,
  'total_tokens': 218,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 0,
   'audio_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0}},
 'service_tier': 'default',
 'system_fingerprint':